In [32]:
"""
INFORMATION RETRIEVAL SYSTEM - PHASE 1: DATA PREPROCESSING
============================================================
This is the foundation of our IR system. We'll load and preprocess the news articles.

Author: Saad Ali
Course: Information Retrieval
"""

import pandas as pd
import numpy as np
import re
import string
from collections import defaultdict, Counter
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Download required NLTK data (run once)
print("Checking and downloading required NLTK resources...")

# Download punkt tokenizer
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    print("Downloading punkt...")
    nltk.download('punkt')

# Download punkt_tab (newer version)
try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    print("Downloading punkt_tab...")
    nltk.download('punkt_tab')
    
# Download stopwords
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    print("Downloading stopwords...")
    nltk.download('stopwords')

print("✓ All NLTK resources ready")

class DocumentPreprocessor:
    """
    This class handles all text preprocessing tasks.
    
    Why do we need preprocessing?
    - Raw text contains noise (punctuation, special characters)
    - Common words like "the", "is" don't help in retrieval
    - Different forms of words (running, runs, ran) should be treated similarly
    """
    
    def __init__(self):
        # Initialize stopwords (common words to remove)
        self.stop_words = set(stopwords.words('english'))
        
        # Initialize stemmer (reduces words to their root form)
        # Example: "running" -> "run", "better" -> "better"
        self.stemmer = PorterStemmer()
        
        print("✓ Preprocessor initialized")
        print(f"✓ Loaded {len(self.stop_words)} stopwords")
    
    def clean_text(self, text):
        """
        Step 1: Clean the raw text
        - Convert to lowercase
        - Remove special characters
        - Remove extra whitespace
        """
        if not isinstance(text, str):
            return ""
        
        # Convert to lowercase
        text = text.lower()
        
        # Remove URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)
        
        # Remove email addresses
        text = re.sub(r'\S+@\S+', '', text)
        
        # Remove special characters and digits
        text = re.sub(r'[^a-zA-Z\s]', ' ', text)
        
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text).strip()
        
        return text
    
    def tokenize(self, text):
        """
        Step 2: Split text into individual words (tokens)
        Example: "I love IR" -> ["I", "love", "IR"]
        """
        return word_tokenize(text)
    
    def remove_stopwords(self, tokens):
        """
        Step 3: Remove common words that don't add meaning
        Example: ["the", "quick", "brown", "fox"] -> ["quick", "brown", "fox"]
        """
        return [word for word in tokens if word not in self.stop_words]
    
    def stem_tokens(self, tokens):
        """
        Step 4: Reduce words to their root form
        Example: ["running", "runs", "ran"] -> ["run", "run", "ran"]
        """
        return [self.stemmer.stem(word) for word in tokens]
    
    def preprocess(self, text):
        """
        Complete preprocessing pipeline
        Combines all steps: clean -> tokenize -> remove stopwords -> stem
        """
        # Step 1: Clean
        cleaned = self.clean_text(text)
        
        # Step 2: Tokenize
        tokens = self.tokenize(cleaned)
        
        # Step 3: Remove stopwords
        tokens = self.remove_stopwords(tokens)
        
        # Step 4: Stem
        tokens = self.stem_tokens(tokens)
        
        # Filter out very short tokens (less than 2 characters)
        tokens = [token for token in tokens if len(token) > 1]
        
        return tokens


class DocumentCollection:
    """
    This class manages the entire document collection.
    It loads, preprocesses, and stores all documents.
    """
    
    def __init__(self, csv_path):
        """
        Initialize and load the document collection
        
        Args:
            csv_path: Path to your CSV file containing news articles
        """
        self.preprocessor = DocumentPreprocessor()
        self.documents = []  # Stores original documents
        self.processed_docs = []  # Stores preprocessed documents
        self.doc_ids = []  # Document identifiers
        
        print("\n" + "="*60)
        print("LOADING DOCUMENT COLLECTION")
        print("="*60)
        
        # Load the CSV file
        self.load_documents(csv_path)
        
    def load_documents(self, csv_path):
        """
        Load documents from CSV file with encoding handling
        """
        try:
            # Try multiple encodings to handle different file formats
            encodings_to_try = ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252', 'utf-16']
            df = None
            
            for encoding in encodings_to_try:
                try:
                    print(f"Trying encoding: {encoding}")
                    df = pd.read_csv(csv_path, encoding=encoding)
                    print(f"✓ Successfully loaded with {encoding} encoding")
                    break
                except UnicodeDecodeError:
                    continue
                except Exception as e:
                    if encoding == encodings_to_try[-1]:
                        raise e
                    continue
            
            if df is None:
                raise ValueError("Could not read file with any supported encoding")
            
            print(f"\n✓ Loaded {len(df)} documents from CSV")
            print(f"✓ Columns available: {df.columns.tolist()}")
            
            # Build document text from available columns
            for idx, row in df.iterrows():
                doc_text = ""
                
                # Check for heading/title column
                if 'Heading' in df.columns:
                    doc_text += str(row['Heading']) + " "
                elif 'heading' in df.columns:
                    doc_text += str(row['heading']) + " "
                elif 'title' in df.columns:
                    doc_text += str(row['title']) + " "
                
                # Check for article/content column
                if 'Article' in df.columns:
                    doc_text += str(row['Article'])
                elif 'article' in df.columns:
                    doc_text += str(row['article'])
                elif 'content' in df.columns:
                    doc_text += str(row['content'])
                elif 'text' in df.columns:
                    doc_text += str(row['text'])
                
                # Store original document
                self.documents.append({
                    'id': idx,
                    'text': doc_text,
                    'original': row.to_dict()
                })
                
                self.doc_ids.append(idx)
            
            print(f"\n✓ Stored {len(self.documents)} documents")
            
            # Preprocess all documents
            self.preprocess_collection()
            
        except FileNotFoundError:
            print(f"✗ Error: File not found at {csv_path}")
            print("Please make sure you've downloaded the dataset from Kaggle")
            print("Check that the file path is correct")
        except Exception as e:
            print(f"✗ Error loading documents: {e}")
            print("\nTroubleshooting tips:")
            print("1. Check if the file path is correct")
            print("2. Make sure the CSV file is not corrupted")
            print("3. Try opening the file in a text editor to check its format")
    
    def preprocess_collection(self):
        """
        Preprocess all documents in the collection
        This is done once and stored for efficiency
        """
        print("\n" + "-"*60)
        print("PREPROCESSING DOCUMENTS")
        print("-"*60)
        
        for idx, doc in enumerate(self.documents):
            # Preprocess the document text
            processed = self.preprocessor.preprocess(doc['text'])
            
            self.processed_docs.append({
                'id': doc['id'],
                'tokens': processed,
                'token_count': len(processed)
            })
            
            # Show progress for every 100 documents
            if (idx + 1) % 100 == 0:
                print(f"Processed {idx + 1}/{len(self.documents)} documents...")
        
        print(f"\n✓ All {len(self.processed_docs)} documents preprocessed!")
        
        # Calculate and display statistics
        self.display_statistics()
    
    def display_statistics(self):
        """
        Display useful statistics about the collection
        """
        print("\n" + "="*60)
        print("COLLECTION STATISTICS")
        print("="*60)
        
        # Calculate average document length
        avg_length = np.mean([doc['token_count'] for doc in self.processed_docs])
        
        # Calculate vocabulary size (unique terms)
        vocabulary = set()
        for doc in self.processed_docs:
            vocabulary.update(doc['tokens'])
        
        print(f"Total Documents: {len(self.documents)}")
        print(f"Average Document Length: {avg_length:.2f} tokens")
        print(f"Vocabulary Size: {len(vocabulary)} unique terms")
        
        # Show example of preprocessing
        if len(self.documents) > 0:
            print("\n" + "-"*60)
            print("PREPROCESSING EXAMPLE")
            print("-"*60)
            example_text = self.documents[0]['text'][:200]
            example_tokens = self.processed_docs[0]['tokens'][:20]
            
            print(f"Original (first 200 chars):\n{example_text}...")
            print(f"\nAfter preprocessing (first 20 tokens):\n{example_tokens}")
    
    def get_document(self, doc_id):
        """Get original document by ID"""
        return self.documents[doc_id]
    
    def get_processed_document(self, doc_id):
        """Get preprocessed document by ID"""
        return self.processed_docs[doc_id]


# ============================================================
# USAGE EXAMPLE
# ============================================================

if __name__ == "__main__":
    """
    HOW TO USE THIS CODE:
    
    1. Download the dataset from Kaggle
    2. Place the CSV file in the same directory as this script
    3. Update the file path below
    4. Run this script
    """
    
    # Path to your CSV file
    CSV_FILE_PATH = r"C:\Users\lenovo\news_articles.csv"  
    
    print("\n" + "="*60)
    print("INFORMATION RETRIEVAL SYSTEM - PHASE 1")
    print("DATA LOADING AND PREPROCESSING")
    print("="*60)
    
    # Create the document collection
    collection = DocumentCollection(CSV_FILE_PATH)
    
    print("\n" + "="*60)
    print("PHASE 1 COMPLETE!")
    print("="*60)
    print("\nYou now have:")
    print("1. Loaded all news articles")
    print("2. Cleaned and preprocessed the text")
    print("3. Created tokens (individual words)")
    print("4. Removed stopwords and stemmed words")
    print("\nNext Phase: We'll implement the Boolean Retrieval Model")
    print("="*60)

Checking and downloading required NLTK resources...
✓ All NLTK resources ready

INFORMATION RETRIEVAL SYSTEM - PHASE 1
DATA LOADING AND PREPROCESSING
✓ Preprocessor initialized
✓ Loaded 198 stopwords

LOADING DOCUMENT COLLECTION
Trying encoding: utf-8
Trying encoding: latin-1
✓ Successfully loaded with latin-1 encoding

✓ Loaded 2692 documents from CSV
✓ Columns available: ['Article', 'Date', 'Heading', 'NewsType']

✓ Stored 2692 documents

------------------------------------------------------------
PREPROCESSING DOCUMENTS
------------------------------------------------------------
Processed 100/2692 documents...
Processed 200/2692 documents...
Processed 300/2692 documents...
Processed 400/2692 documents...
Processed 500/2692 documents...
Processed 600/2692 documents...
Processed 700/2692 documents...
Processed 800/2692 documents...
Processed 900/2692 documents...
Processed 1000/2692 documents...
Processed 1100/2692 documents...
Processed 1200/2692 documents...
Processed 1300/2692 do

In [34]:
"""
INFORMATION RETRIEVAL SYSTEM - PHASE 2: BOOLEAN RETRIEVAL MODEL
================================================================
This implements the Boolean Retrieval Model using an inverted index.

What is Boolean Retrieval?
- Uses AND, OR, NOT operators to find documents
- Example: "election AND politics" finds docs with BOTH terms
- Fast and precise, but no ranking (results are binary: match or no match)

Author: Saad Ali
Course: Information Retrieval
"""

import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from typing import List, Set, Dict

# Download required NLTK data (if not already downloaded)
print("Checking NLTK resources...")
try:
    nltk.data.find('tokenizers/punkt_tab')
    nltk.data.find('corpora/stopwords')
    print("✓ NLTK resources ready")
except LookupError:
    print("Downloading required NLTK data...")
    nltk.download('punkt_tab')
    nltk.download('stopwords')


# ============================================================
# PHASE 1 CLASSES (Embedded for convenience)
# ============================================================

class DocumentPreprocessor:
    """Handles all text preprocessing tasks"""
    
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        self.stemmer = PorterStemmer()
    
    def clean_text(self, text):
        if not isinstance(text, str):
            return ""
        text = text.lower()
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)
        text = re.sub(r'\S+@\S+', '', text)
        text = re.sub(r'[^a-zA-Z\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    
    def tokenize(self, text):
        return word_tokenize(text)
    
    def remove_stopwords(self, tokens):
        return [word for word in tokens if word not in self.stop_words]
    
    def stem_tokens(self, tokens):
        return [self.stemmer.stem(word) for word in tokens]
    
    def preprocess(self, text):
        cleaned = self.clean_text(text)
        tokens = self.tokenize(cleaned)
        tokens = self.remove_stopwords(tokens)
        tokens = self.stem_tokens(tokens)
        tokens = [token for token in tokens if len(token) > 1]
        return tokens


class DocumentCollection:
    """Manages the entire document collection"""
    
    def __init__(self, csv_path):
        self.preprocessor = DocumentPreprocessor()
        self.documents = []
        self.processed_docs = []
        self.doc_ids = []
        self.load_documents(csv_path)
        
    def load_documents(self, csv_path):
        try:
            # Try multiple encodings
            encodings_to_try = ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252']
            df = None
            
            for encoding in encodings_to_try:
                try:
                    df = pd.read_csv(csv_path, encoding=encoding)
                    print(f"✓ Loaded with {encoding} encoding")
                    break
                except UnicodeDecodeError:
                    continue
            
            if df is None:
                raise ValueError("Could not read file with any supported encoding")
            
            print(f"✓ Loaded {len(df)} documents")
            
            # Build documents from CSV
            for idx, row in df.iterrows():
                doc_text = ""
                
                # Check for heading/title
                if 'Heading' in df.columns:
                    doc_text += str(row['Heading']) + " "
                elif 'heading' in df.columns:
                    doc_text += str(row['heading']) + " "
                elif 'title' in df.columns:
                    doc_text += str(row['title']) + " "
                
                # Check for article/content
                if 'Article' in df.columns:
                    doc_text += str(row['Article'])
                elif 'article' in df.columns:
                    doc_text += str(row['article'])
                elif 'content' in df.columns:
                    doc_text += str(row['content'])
                elif 'text' in df.columns:
                    doc_text += str(row['text'])
                
                self.documents.append({
                    'id': idx,
                    'text': doc_text,
                    'original': row.to_dict()
                })
                self.doc_ids.append(idx)
            
            # Preprocess all documents
            self.preprocess_collection()
            
        except Exception as e:
            print(f"✗ Error loading documents: {e}")
    
    def preprocess_collection(self):
        print("\nPreprocessing documents...")
        for idx, doc in enumerate(self.documents):
            processed = self.preprocessor.preprocess(doc['text'])
            self.processed_docs.append({
                'id': doc['id'],
                'tokens': processed,
                'token_count': len(processed)
            })
            
            if (idx + 1) % 500 == 0:
                print(f"  Processed {idx + 1}/{len(self.documents)} documents...")
        
        print(f"✓ All {len(self.processed_docs)} documents preprocessed!")
    
    def get_document(self, doc_id):
        return self.documents[doc_id]
    
    def get_processed_document(self, doc_id):
        return self.processed_docs[doc_id]


# ============================================================
# PHASE 2: BOOLEAN RETRIEVAL MODEL
# ============================================================

class InvertedIndex:
    """
    An Inverted Index is the core data structure for efficient retrieval.
    
    What is an Inverted Index?
    ----------------------------
    Instead of storing "which words are in each document",
    we store "which documents contain each word".
    
    Example:
    Doc1: "cat dog"
    Doc2: "dog bird"
    Doc3: "cat bird"
    
    Inverted Index:
    "cat" -> [Doc1, Doc3]
    "dog" -> [Doc1, Doc2]
    "bird" -> [Doc2, Doc3]
    """
    
    def __init__(self):
        self.index = defaultdict(set)
        self.term_doc_freq = defaultdict(int)
        self.num_documents = 0
        print("\n✓ Inverted Index initialized")
    
    def build_index(self, processed_docs: List[Dict]):
        """Build the inverted index from preprocessed documents"""
        print("\n" + "="*60)
        print("BUILDING INVERTED INDEX")
        print("="*60)
        
        self.num_documents = len(processed_docs)
        
        for doc in processed_docs:
            doc_id = doc['id']
            tokens = doc['tokens']
            
            # Get unique terms in this document
            unique_terms = set(tokens)
            
            # For each unique term, add this document to its posting list
            for term in unique_terms:
                self.index[term].add(doc_id)
                self.term_doc_freq[term] += 1
            
            if (doc_id + 1) % 500 == 0:
                print(f"  Indexed {doc_id + 1}/{self.num_documents} documents...")
        
        print(f"\n✓ Inverted index built successfully!")
        print(f"✓ Indexed {len(self.index)} unique terms")
        
        self.display_sample_entries()
    
    def display_sample_entries(self):
        """Display sample inverted index entries"""
        print("\n" + "-"*60)
        print("SAMPLE INVERTED INDEX ENTRIES")
        print("-"*60)
        
        sample_terms = list(self.index.keys())[:5]
        
        for term in sample_terms:
            doc_ids = list(self.index[term])[:5]
            print(f"\nTerm: '{term}'")
            print(f"  Appears in {len(self.index[term])} documents")
            print(f"  Sample doc IDs: {doc_ids}...")
    
    def get_postings(self, term: str) -> Set[int]:
        """Get the posting list for a term"""
        return self.index.get(term, set())
    
    def get_term_frequency(self, term: str) -> int:
        """Get document frequency of a term"""
        return self.term_doc_freq.get(term, 0)


class BooleanRetrievalModel:
    """
    Implements Boolean Retrieval with AND, OR, NOT operators
    
    Boolean Operators:
    ------------------
    - AND: Documents must contain ALL terms (intersection)
    - OR: Documents must contain AT LEAST ONE term (union)
    - NOT: Exclude documents containing the term (difference)
    """
    
    def __init__(self, inverted_index: InvertedIndex, preprocessor: DocumentPreprocessor):
        self.index = inverted_index
        self.preprocessor = preprocessor
        print("\n✓ Boolean Retrieval Model initialized")
    
    def parse_query(self, query: str) -> List[str]:
        """Parse and preprocess the query"""
        tokens = []
        parts = query.split()
        
        for part in parts:
            upper_part = part.upper()
            if upper_part in ['AND', 'OR', 'NOT']:
                tokens.append(upper_part)
            else:
                processed_terms = self.preprocessor.preprocess(part)
                tokens.extend(processed_terms)
        
        return tokens
    
    def execute_query(self, query: str) -> Set[int]:
        """Execute a Boolean query and return matching document IDs"""
        tokens = self.parse_query(query)
        
        if not tokens:
            return set()
        
        if 'AND' in tokens:
            return self._execute_and_query(tokens)
        elif 'OR' in tokens:
            return self._execute_or_query(tokens)
        elif 'NOT' in tokens:
            return self._execute_not_query(tokens)
        else:
            return self._execute_or_query(tokens)
    
    def _execute_and_query(self, tokens: List[str]) -> Set[int]:
        """Execute AND query (intersection)"""
        term_groups = []
        current_group = []
        
        for token in tokens:
            if token == 'AND':
                if current_group:
                    term_groups.append(current_group)
                    current_group = []
            else:
                current_group.append(token)
        
        if current_group:
            term_groups.append(current_group)
        
        result = None
        
        for group in term_groups:
            group_result = set()
            for term in group:
                group_result = group_result.union(self.index.get_postings(term))
            
            if result is None:
                result = group_result
            else:
                result = result.intersection(group_result)
        
        return result if result else set()
    
    def _execute_or_query(self, tokens: List[str]) -> Set[int]:
        """Execute OR query (union)"""
        result = set()
        
        for token in tokens:
            if token not in ['OR', 'AND', 'NOT']:
                result = result.union(self.index.get_postings(token))
        
        return result
    
    def _execute_not_query(self, tokens: List[str]) -> Set[int]:
        """Execute NOT query (difference)"""
        not_index = tokens.index('NOT')
        
        include_terms = [t for t in tokens[:not_index] if t not in ['AND', 'OR', 'NOT']]
        exclude_terms = [t for t in tokens[not_index+1:] if t not in ['AND', 'OR', 'NOT']]
        
        result = set()
        for term in include_terms:
            result = result.union(self.index.get_postings(term))
        
        if not result:
            result = set(range(self.index.num_documents))
        
        for term in exclude_terms:
            result = result.difference(self.index.get_postings(term))
        
        return result
    
    def search(self, query: str, collection: DocumentCollection, top_k: int = 10) -> List[Dict]:
        """Perform a search and return results"""
        print("\n" + "="*60)
        print(f"EXECUTING BOOLEAN QUERY: '{query}'")
        print("="*60)
        
        matching_doc_ids = self.execute_query(query)
        
        print(f"\n✓ Found {len(matching_doc_ids)} matching documents")
        
        results = []
        for doc_id in list(matching_doc_ids)[:top_k]:
            doc = collection.get_document(doc_id)
            
            # Get title from original data
            title = "N/A"
            if 'Heading' in doc['original']:
                title = doc['original']['Heading']
            elif 'heading' in doc['original']:
                title = doc['original']['heading']
            elif 'title' in doc['original']:
                title = doc['original']['title']
            
            results.append({
                'doc_id': doc_id,
                'title': title,
                'text_preview': doc['text'][:200] + "...",
                'relevance': 'MATCH'
            })
        
        return results


# ============================================================
# USAGE EXAMPLE AND DEMONSTRATION
# ============================================================

def demonstrate_boolean_retrieval():
    """Complete demonstration of Boolean Retrieval Model"""
    print("\n" + "="*60)
    print("INFORMATION RETRIEVAL SYSTEM - PHASE 2")
    print("BOOLEAN RETRIEVAL MODEL")
    print("="*60)
    
    # Step 1: Load and preprocess documents
    print("\nStep 1: Loading documents...")
    CSV_FILE_PATH = r"C:\Users\lenovo\news_articles.csv"  # Update this path
    collection = DocumentCollection(CSV_FILE_PATH)
    
    # Step 2: Build inverted index
    print("\nStep 2: Building inverted index...")
    inverted_index = InvertedIndex()
    inverted_index.build_index(collection.processed_docs)
    
    # Step 3: Initialize Boolean Retrieval Model
    print("\nStep 3: Initializing Boolean Retrieval Model...")
    boolean_model = BooleanRetrievalModel(inverted_index, collection.preprocessor)
    
    # Step 4: Demonstrate different types of queries
    print("\n" + "="*60)
    print("QUERY DEMONSTRATIONS")
    print("="*60)
    
    # Example queries (adjust based on your dataset)
    queries = [
        "karachi",                           # Single term
        "karachi AND transport",             # AND query
        "karachi OR lahore",                 # OR query
        "government NOT corruption",         # NOT query
        "pakistan AND election"              # Multiple terms with AND
    ]
    
    for query in queries:
        results = boolean_model.search(query, collection, top_k=5)
        
        print("\n" + "-"*60)
        print(f"Query: '{query}'")
        print(f"Results: {len(results)} documents shown (top 5)")
        print("-"*60)
        
        for i, result in enumerate(results, 1):
            print(f"\n{i}. Document ID: {result['doc_id']}")
            print(f"   Title: {result['title']}")
            print(f"   Preview: {result['text_preview']}")
    
    print("\n" + "="*60)
    print("PHASE 2 COMPLETE!")
    print("="*60)
    print("\nKey Achievements:")
    print("✓ Built an inverted index for fast retrieval")
    print("✓ Implemented Boolean operators (AND, OR, NOT)")
    print("✓ Demonstrated different query types")
    print("\nLimitations of Boolean Model:")
    print("- No ranking (all results are equally relevant)")
    print("- Exact matching only (no partial matches)")
    print("- Can return too many or too few results")
    print("\nNext Phase: TF-IDF Model (with ranking!)")
    print("="*60)


if __name__ == "__main__":
    demonstrate_boolean_retrieval()

Checking NLTK resources...
✓ NLTK resources ready

INFORMATION RETRIEVAL SYSTEM - PHASE 2
BOOLEAN RETRIEVAL MODEL

Step 1: Loading documents...
✓ Loaded with latin-1 encoding
✓ Loaded 2692 documents

Preprocessing documents...
  Processed 500/2692 documents...
  Processed 1000/2692 documents...
  Processed 1500/2692 documents...
  Processed 2000/2692 documents...
  Processed 2500/2692 documents...
✓ All 2692 documents preprocessed!

Step 2: Building inverted index...

✓ Inverted Index initialized

BUILDING INVERTED INDEX
  Indexed 500/2692 documents...
  Indexed 1000/2692 documents...
  Indexed 1500/2692 documents...
  Indexed 2000/2692 documents...
  Indexed 2500/2692 documents...

✓ Inverted index built successfully!
✓ Indexed 18196 unique terms

------------------------------------------------------------
SAMPLE INVERTED INDEX ENTRIES
------------------------------------------------------------

Term: 'public'
  Appears in 172 documents
  Sample doc IDs: [0, 1, 2562, 4, 2566]...

Te

In [4]:
"""
INFORMATION RETRIEVAL SYSTEM - PHASE 3: TF-IDF RETRIEVAL MODEL
===============================================================
This implements the TF-IDF (Term Frequency-Inverse Document Frequency) Model.

What is TF-IDF?
---------------
TF-IDF is a numerical statistic that reflects how important a word is to a 
document in a collection. It combines two concepts:

1. TF (Term Frequency): How often does a term appear in THIS document?
2. IDF (Inverse Document Frequency): How rare is this term across ALL documents?

Key Innovation: TF-IDF provides RANKING - documents are scored and ordered!

Author: Saad Ali
Course: Information Retrieval
"""

import pandas as pd
import numpy as np
import math
import re
import nltk
from collections import defaultdict, Counter
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from typing import List, Dict, Tuple, Set

# Download required NLTK data
print("Checking NLTK resources...")
try:
    nltk.data.find('tokenizers/punkt_tab')
    nltk.data.find('corpora/stopwords')
    print("✓ NLTK resources ready")
except LookupError:
    print("Downloading required NLTK data...")
    nltk.download('punkt_tab')
    nltk.download('stopwords')


# ============================================================
# PHASE 1 & 2 CLASSES (Embedded for convenience)
# ============================================================

class DocumentPreprocessor:
    """Handles all text preprocessing tasks"""
    
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        self.stemmer = PorterStemmer()
    
    def clean_text(self, text):
        if not isinstance(text, str):
            return ""
        text = text.lower()
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)
        text = re.sub(r'\S+@\S+', '', text)
        text = re.sub(r'[^a-zA-Z\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    
    def tokenize(self, text):
        return word_tokenize(text)
    
    def remove_stopwords(self, tokens):
        return [word for word in tokens if word not in self.stop_words]
    
    def stem_tokens(self, tokens):
        return [self.stemmer.stem(word) for word in tokens]
    
    def preprocess(self, text):
        cleaned = self.clean_text(text)
        tokens = self.tokenize(cleaned)
        tokens = self.remove_stopwords(tokens)
        tokens = self.stem_tokens(tokens)
        tokens = [token for token in tokens if len(token) > 1]
        return tokens


class DocumentCollection:
    """Manages the entire document collection"""
    
    def __init__(self, csv_path):
        self.preprocessor = DocumentPreprocessor()
        self.documents = []
        self.processed_docs = []
        self.doc_ids = []
        self.load_documents(csv_path)
        
    def load_documents(self, csv_path):
        try:
            encodings_to_try = ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252']
            df = None
            
            for encoding in encodings_to_try:
                try:
                    df = pd.read_csv(csv_path, encoding=encoding)
                    print(f"✓ Loaded with {encoding} encoding")
                    break
                except UnicodeDecodeError:
                    continue
            
            if df is None:
                raise ValueError("Could not read file")
            
            print(f"✓ Loaded {len(df)} documents")
            
            for idx, row in df.iterrows():
                doc_text = ""
                
                if 'Heading' in df.columns:
                    doc_text += str(row['Heading']) + " "
                elif 'heading' in df.columns:
                    doc_text += str(row['heading']) + " "
                elif 'title' in df.columns:
                    doc_text += str(row['title']) + " "
                
                if 'Article' in df.columns:
                    doc_text += str(row['Article'])
                elif 'article' in df.columns:
                    doc_text += str(row['article'])
                elif 'content' in df.columns:
                    doc_text += str(row['content'])
                elif 'text' in df.columns:
                    doc_text += str(row['text'])
                
                self.documents.append({
                    'id': idx,
                    'text': doc_text,
                    'original': row.to_dict()
                })
                self.doc_ids.append(idx)
            
            self.preprocess_collection()
            
        except Exception as e:
            print(f"✗ Error loading documents: {e}")
    
    def preprocess_collection(self):
        print("\nPreprocessing documents...")
        for idx, doc in enumerate(self.documents):
            processed = self.preprocessor.preprocess(doc['text'])
            self.processed_docs.append({
                'id': doc['id'],
                'tokens': processed,
                'token_count': len(processed)
            })
            
            if (idx + 1) % 500 == 0:
                print(f"  Processed {idx + 1}/{len(self.documents)} documents...")
        
        print(f"✓ All {len(self.processed_docs)} documents preprocessed!")
    
    def get_document(self, doc_id):
        return self.documents[doc_id]
    
    def get_processed_document(self, doc_id):
        return self.processed_docs[doc_id]


class InvertedIndex:
    """Inverted Index for efficient retrieval"""
    
    def __init__(self):
        self.index = defaultdict(set)
        self.term_doc_freq = defaultdict(int)
        self.num_documents = 0
    
    def build_index(self, processed_docs: List[Dict]):
        """Build the inverted index"""
        print("\nBuilding inverted index...")
        
        self.num_documents = len(processed_docs)
        
        for doc in processed_docs:
            doc_id = doc['id']
            tokens = doc['tokens']
            unique_terms = set(tokens)
            
            for term in unique_terms:
                self.index[term].add(doc_id)
                self.term_doc_freq[term] += 1
            
            if (doc_id + 1) % 500 == 0:
                print(f"  Indexed {doc_id + 1}/{self.num_documents} documents...")
        
        print(f"✓ Indexed {len(self.index)} unique terms")
    
    def get_postings(self, term: str) -> Set[int]:
        return self.index.get(term, set())
    
    def get_term_frequency(self, term: str) -> int:
        return self.term_doc_freq.get(term, 0)


# ============================================================
# PHASE 3: TF-IDF MODEL
# ============================================================

class TFIDFModel:
    """
    Implements TF-IDF Retrieval Model with Ranking
    
    Mathematical Formulas:
    ----------------------
    1. Term Frequency (TF):
       TF(t, d) = (Number of times term t appears in document d) / 
                  (Total number of terms in document d)
    
    2. Inverse Document Frequency (IDF):
       IDF(t) = log(Total documents / Documents containing term t)
    
    3. TF-IDF Score:
       TF-IDF(t, d) = TF(t, d) × IDF(t)
    
    4. Document Score (for a query):
       Score(q, d) = Σ TF-IDF(t, d) for all terms t in query q
    """
    
    def __init__(self, collection: DocumentCollection, inverted_index: InvertedIndex):
        """Initialize TF-IDF Model"""
        self.collection = collection
        self.index = inverted_index
        self.preprocessor = collection.preprocessor
        
        # TF-IDF matrix: {doc_id: {term: tf_idf_score}}
        self.tfidf_matrix = {}
        
        # IDF values: {term: idf_value}
        self.idf_values = {}
        
        # Term frequencies: {doc_id: {term: count}}
        self.term_frequencies = {}
        
        print("\n✓ TF-IDF Model initialized")
        self.build_tfidf_matrix()
    
    def calculate_tf(self, term_count: int, total_terms: int, method: str = 'normalized') -> float:
        """
        Calculate Term Frequency (TF)
        
        Methods:
        - 'raw': Just the count
        - 'normalized': count / total_terms (most common)
        - 'log': 1 + log(count) if count > 0
        """
        if term_count == 0:
            return 0.0
        
        if method == 'raw':
            return float(term_count)
        elif method == 'normalized':
            return term_count / total_terms if total_terms > 0 else 0.0
        elif method == 'log':
            return 1.0 + math.log(term_count)
        else:
            return term_count / total_terms if total_terms > 0 else 0.0
    
    def calculate_idf(self, term: str) -> float:
        """
        Calculate Inverse Document Frequency (IDF)
        
        Formula: IDF(t) = log(N / df(t))
        Where:
        - N = total number of documents
        - df(t) = number of documents containing term t
        """
        N = self.index.num_documents
        df = self.index.get_term_frequency(term)
        
        if df == 0:
            return 0.0
        
        # Add 1 to denominator for smoothing
        idf = math.log(N / (1 + df))
        
        return idf
    
    def build_tfidf_matrix(self):
        """Build TF-IDF matrix for all documents"""
        print("\n" + "="*60)
        print("BUILDING TF-IDF MATRIX")
        print("="*60)
        
        # Step 1: Calculate term frequencies
        print("\nStep 1: Calculating term frequencies...")
        for doc in self.collection.processed_docs:
            doc_id = doc['id']
            tokens = doc['tokens']
            term_freq = Counter(tokens)
            self.term_frequencies[doc_id] = term_freq
            
            if (doc_id + 1) % 500 == 0:
                print(f"  Processed {doc_id + 1}/{len(self.collection.processed_docs)} documents...")
        
        # Step 2: Calculate IDF values
        print("\nStep 2: Calculating IDF values...")
        all_terms = self.index.index.keys()
        for term in all_terms:
            self.idf_values[term] = self.calculate_idf(term)
        
        print(f"✓ Calculated IDF for {len(self.idf_values)} unique terms")
        
        # Step 3: Calculate TF-IDF scores
        print("\nStep 3: Calculating TF-IDF scores...")
        for doc in self.collection.processed_docs:
            doc_id = doc['id']
            total_terms = doc['token_count']
            
            self.tfidf_matrix[doc_id] = {}
            
            for term, count in self.term_frequencies[doc_id].items():
                tf = self.calculate_tf(count, total_terms, method='normalized')
                idf = self.idf_values.get(term, 0.0)
                tfidf = tf * idf
                
                self.tfidf_matrix[doc_id][term] = tfidf
            
            if (doc_id + 1) % 500 == 0:
                print(f"  Calculated TF-IDF for {doc_id + 1}/{len(self.collection.processed_docs)} documents...")
        
        print("\n✓ TF-IDF matrix built successfully!")
        self.display_tfidf_statistics()
    
    def display_tfidf_statistics(self):
        """Display TF-IDF statistics"""
        print("\n" + "="*60)
        print("TF-IDF STATISTICS")
        print("="*60)
        
        sorted_idf = sorted(self.idf_values.items(), key=lambda x: x[1], reverse=True)
        
        print("\nTop 10 Most Discriminative Terms (Highest IDF):")
        print("-" * 60)
        for i, (term, idf) in enumerate(sorted_idf[:10], 1):
            doc_count = self.index.get_term_frequency(term)
            print(f"{i:2d}. '{term}' - IDF: {idf:.4f} (in {doc_count} docs)")
        
        print("\nTop 10 Most Common Terms (Lowest IDF):")
        print("-" * 60)
        for i, (term, idf) in enumerate(sorted_idf[-10:][::-1], 1):
            doc_count = self.index.get_term_frequency(term)
            print(f"{i:2d}. '{term}' - IDF: {idf:.4f} (in {doc_count} docs)")
    
    def calculate_query_tfidf(self, query_terms: List[str]) -> Dict[str, float]:
        """Calculate TF-IDF for query terms"""
        query_tfidf = {}
        query_tf = Counter(query_terms)
        total_query_terms = len(query_terms)
        
        for term, count in query_tf.items():
            tf = self.calculate_tf(count, total_query_terms, method='normalized')
            idf = self.idf_values.get(term, 0.0)
            query_tfidf[term] = tf * idf
        
        return query_tfidf
    
    def calculate_similarity(self, query_tfidf: Dict[str, float], doc_id: int) -> float:
        """
        Calculate similarity between query and document
        Uses dot product of TF-IDF vectors
        """
        score = 0.0
        doc_tfidf = self.tfidf_matrix.get(doc_id, {})
        
        for term, query_score in query_tfidf.items():
            doc_score = doc_tfidf.get(term, 0.0)
            score += query_score * doc_score
        
        return score
    
    def search(self, query: str, top_k: int = 10) -> List[Dict]:
        """
        Search for documents using TF-IDF ranking
        
        Returns ranked list of documents with scores
        """
        print("\n" + "="*60)
        print(f"EXECUTING TF-IDF QUERY: '{query}'")
        print("="*60)
        
        # Preprocess query
        query_terms = self.preprocessor.preprocess(query)
        print(f"\nPreprocessed query: {query_terms}")
        
        if not query_terms:
            print("✗ No valid query terms")
            return []
        
        # Calculate query TF-IDF
        query_tfidf = self.calculate_query_tfidf(query_terms)
        
        print("\nQuery TF-IDF scores:")
        for term, score in query_tfidf.items():
            print(f"  '{term}': {score:.4f}")
        
        # Calculate similarity for all documents
        print("\nRanking documents...")
        doc_scores = []
        
        for doc_id in range(len(self.collection.documents)):
            score = self.calculate_similarity(query_tfidf, doc_id)
            if score > 0:
                doc_scores.append((doc_id, score))
        
        # Sort by score
        doc_scores.sort(key=lambda x: x[1], reverse=True)
        
        print(f"✓ Found {len(doc_scores)} relevant documents")
        
        # Get top-k results
        results = []
        for doc_id, score in doc_scores[:top_k]:
            doc = self.collection.get_document(doc_id)
            
            # Get title
            title = "N/A"
            if 'Heading' in doc['original']:
                title = doc['original']['Heading']
            elif 'heading' in doc['original']:
                title = doc['original']['heading']
            elif 'title' in doc['original']:
                title = doc['original']['title']
            
            results.append({
                'rank': len(results) + 1,
                'doc_id': doc_id,
                'score': score,
                'title': title,
                'text_preview': doc['text'][:200] + "..."
            })
        
        return results


# ============================================================
# DEMONSTRATION
# ============================================================

def demonstrate_tfidf():
    """Complete demonstration of TF-IDF Model"""
    print("\n" + "="*60)
    print("INFORMATION RETRIEVAL SYSTEM - PHASE 3")
    print("TF-IDF RETRIEVAL MODEL")
    print("="*60)
    
    # Load documents
    print("\nStep 1: Loading documents...")
    CSV_FILE_PATH = r"C:\Users\lenovo\news_articles.csv"
    collection = DocumentCollection(CSV_FILE_PATH)
    
    # Build inverted index
    print("\nStep 2: Building inverted index...")
    inverted_index = InvertedIndex()
    inverted_index.build_index(collection.processed_docs)
    
    # Initialize TF-IDF Model
    print("\nStep 3: Initializing TF-IDF Model...")
    tfidf_model = TFIDFModel(collection, inverted_index)
    
    # Demonstrate queries
    print("\n" + "="*60)
    print("QUERY DEMONSTRATIONS")
    print("="*60)
    
    queries = [
        "karachi transport government",
        "pakistan cricket victory",
        "lahore education university",
    ]
    
    for query in queries:
        results = tfidf_model.search(query, top_k=5)
        
        print("\n" + "="*60)
        print(f"Query: '{query}'")
        print("="*60)
        
        if not results:
            print("No results found.")
            continue
        
        for result in results:
            print(f"\n{result['rank']}. [Score: {result['score']:.4f}] Doc ID: {result['doc_id']}")
            print(f"   Title: {result['title']}")
            print(f"   Preview: {result['text_preview']}")
    
    print("\n" + "="*60)
    print("PHASE 3 COMPLETE!")
    print("="*60)
    print("\nKey Achievements:")
    print("✓ Implemented TF-IDF scoring")
    print("✓ Built TF-IDF matrix for all documents")
    print("✓ Implemented document ranking")
    print("\nAdvantages of TF-IDF:")
    print("✓ Ranked results (most relevant first)")
    print("✓ Handles term importance (rare terms weighted higher)")
    print("✓ Better user experience than Boolean model")
    print("✓ More flexible queries")
    print("="*60)


if __name__ == "__main__":
    demonstrate_tfidf()

Checking NLTK resources...
✓ NLTK resources ready

INFORMATION RETRIEVAL SYSTEM - PHASE 3
TF-IDF RETRIEVAL MODEL

Step 1: Loading documents...
✓ Loaded with latin-1 encoding
✓ Loaded 2692 documents

Preprocessing documents...
  Processed 500/2692 documents...
  Processed 1000/2692 documents...
  Processed 1500/2692 documents...
  Processed 2000/2692 documents...
  Processed 2500/2692 documents...
✓ All 2692 documents preprocessed!

Step 2: Building inverted index...

Building inverted index...
  Indexed 500/2692 documents...
  Indexed 1000/2692 documents...
  Indexed 1500/2692 documents...
  Indexed 2000/2692 documents...
  Indexed 2500/2692 documents...
✓ Indexed 18196 unique terms

Step 3: Initializing TF-IDF Model...

✓ TF-IDF Model initialized

BUILDING TF-IDF MATRIX

Step 1: Calculating term frequencies...
  Processed 500/2692 documents...
  Processed 1000/2692 documents...
  Processed 1500/2692 documents...
  Processed 2000/2692 documents...
  Processed 2500/2692 documents...

St

In [38]:
"""
INFORMATION RETRIEVAL SYSTEM - PHASE 4: BM25 RETRIEVAL MODEL
=============================================================
This implements the BM25 (Best Match 25) ranking function.

What is BM25?
-------------
BM25 is a probabilistic ranking function that improves upon TF-IDF.
It's the "gold standard" for text retrieval - used by Elasticsearch and Lucene.

Key Improvements over TF-IDF:
1. SATURATION: Term frequency has diminishing returns
2. DOCUMENT LENGTH NORMALIZATION: Fair comparison
3. TUNABLE PARAMETERS: Customize for your data

Author: Saad Ali
Course: Information Retrieval
"""

import pandas as pd
import numpy as np
import math
import re
import nltk
from collections import defaultdict, Counter
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from typing import List, Dict, Tuple, Set

# Download required NLTK data
print("Checking NLTK resources...")
try:
    nltk.data.find('tokenizers/punkt_tab')
    nltk.data.find('corpora/stopwords')
    print("✓ NLTK resources ready")
except LookupError:
    print("Downloading required NLTK data...")
    nltk.download('punkt_tab')
    nltk.download('stopwords')


# ============================================================
# PHASE 1, 2, 3 CLASSES (Embedded)
# ============================================================

class DocumentPreprocessor:
    """Handles all text preprocessing tasks"""
    
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        self.stemmer = PorterStemmer()
    
    def clean_text(self, text):
        if not isinstance(text, str):
            return ""
        text = text.lower()
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)
        text = re.sub(r'\S+@\S+', '', text)
        text = re.sub(r'[^a-zA-Z\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    
    def tokenize(self, text):
        return word_tokenize(text)
    
    def remove_stopwords(self, tokens):
        return [word for word in tokens if word not in self.stop_words]
    
    def stem_tokens(self, tokens):
        return [self.stemmer.stem(word) for word in tokens]
    
    def preprocess(self, text):
        cleaned = self.clean_text(text)
        tokens = self.tokenize(cleaned)
        tokens = self.remove_stopwords(tokens)
        tokens = self.stem_tokens(tokens)
        tokens = [token for token in tokens if len(token) > 1]
        return tokens


class DocumentCollection:
    """Manages the entire document collection"""
    
    def __init__(self, csv_path):
        self.preprocessor = DocumentPreprocessor()
        self.documents = []
        self.processed_docs = []
        self.doc_ids = []
        self.load_documents(csv_path)
        
    def load_documents(self, csv_path):
        try:
            encodings_to_try = ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252']
            df = None
            
            for encoding in encodings_to_try:
                try:
                    df = pd.read_csv(csv_path, encoding=encoding)
                    print(f"✓ Loaded with {encoding} encoding")
                    break
                except UnicodeDecodeError:
                    continue
            
            if df is None:
                raise ValueError("Could not read file")
            
            print(f"✓ Loaded {len(df)} documents")
            
            for idx, row in df.iterrows():
                doc_text = ""
                
                if 'Heading' in df.columns:
                    doc_text += str(row['Heading']) + " "
                elif 'heading' in df.columns:
                    doc_text += str(row['heading']) + " "
                elif 'title' in df.columns:
                    doc_text += str(row['title']) + " "
                
                if 'Article' in df.columns:
                    doc_text += str(row['Article'])
                elif 'article' in df.columns:
                    doc_text += str(row['article'])
                elif 'content' in df.columns:
                    doc_text += str(row['content'])
                elif 'text' in df.columns:
                    doc_text += str(row['text'])
                
                self.documents.append({
                    'id': idx,
                    'text': doc_text,
                    'original': row.to_dict()
                })
                self.doc_ids.append(idx)
            
            self.preprocess_collection()
            
        except Exception as e:
            print(f"✗ Error loading documents: {e}")
    
    def preprocess_collection(self):
        print("\nPreprocessing documents...")
        for idx, doc in enumerate(self.documents):
            processed = self.preprocessor.preprocess(doc['text'])
            self.processed_docs.append({
                'id': doc['id'],
                'tokens': processed,
                'token_count': len(processed)
            })
            
            if (idx + 1) % 500 == 0:
                print(f"  Processed {idx + 1}/{len(self.documents)} documents...")
        
        print(f"✓ All {len(self.processed_docs)} documents preprocessed!")
    
    def get_document(self, doc_id):
        return self.documents[doc_id]
    
    def get_processed_document(self, doc_id):
        return self.processed_docs[doc_id]


class InvertedIndex:
    """Inverted Index for efficient retrieval"""
    
    def __init__(self):
        self.index = defaultdict(set)
        self.term_doc_freq = defaultdict(int)
        self.num_documents = 0
    
    def build_index(self, processed_docs: List[Dict]):
        """Build the inverted index"""
        print("\nBuilding inverted index...")
        
        self.num_documents = len(processed_docs)
        
        for doc in processed_docs:
            doc_id = doc['id']
            tokens = doc['tokens']
            unique_terms = set(tokens)
            
            for term in unique_terms:
                self.index[term].add(doc_id)
                self.term_doc_freq[term] += 1
            
            if (doc_id + 1) % 500 == 0:
                print(f"  Indexed {doc_id + 1}/{self.num_documents} documents...")
        
        print(f"✓ Indexed {len(self.index)} unique terms")
    
    def get_postings(self, term: str) -> Set[int]:
        return self.index.get(term, set())
    
    def get_term_frequency(self, term: str) -> int:
        return self.term_doc_freq.get(term, 0)


class TFIDFModel:
    """TF-IDF Model (needed for comparison)"""
    
    def __init__(self, collection: DocumentCollection, inverted_index: InvertedIndex):
        self.collection = collection
        self.index = inverted_index
        self.preprocessor = collection.preprocessor
        self.tfidf_matrix = {}
        self.idf_values = {}
        self.term_frequencies = {}
        self.build_tfidf_matrix()
    
    def calculate_tf(self, term_count: int, total_terms: int) -> float:
        if term_count == 0:
            return 0.0
        return term_count / total_terms if total_terms > 0 else 0.0
    
    def calculate_idf(self, term: str) -> float:
        N = self.index.num_documents
        df = self.index.get_term_frequency(term)
        if df == 0:
            return 0.0
        return math.log(N / (1 + df))
    
    def build_tfidf_matrix(self):
        print("\nBuilding TF-IDF matrix (for comparison)...")
        
        for doc in self.collection.processed_docs:
            doc_id = doc['id']
            tokens = doc['tokens']
            term_freq = Counter(tokens)
            self.term_frequencies[doc_id] = term_freq
        
        all_terms = self.index.index.keys()
        for term in all_terms:
            self.idf_values[term] = self.calculate_idf(term)
        
        for doc in self.collection.processed_docs:
            doc_id = doc['id']
            total_terms = doc['token_count']
            self.tfidf_matrix[doc_id] = {}
            
            for term, count in self.term_frequencies[doc_id].items():
                tf = self.calculate_tf(count, total_terms)
                idf = self.idf_values.get(term, 0.0)
                self.tfidf_matrix[doc_id][term] = tf * idf
        
        print("✓ TF-IDF matrix built")
    
    def calculate_query_tfidf(self, query_terms: List[str]) -> Dict[str, float]:
        query_tfidf = {}
        query_tf = Counter(query_terms)
        total_query_terms = len(query_terms)
        
        for term, count in query_tf.items():
            tf = self.calculate_tf(count, total_query_terms)
            idf = self.idf_values.get(term, 0.0)
            query_tfidf[term] = tf * idf
        
        return query_tfidf
    
    def calculate_similarity(self, query_tfidf: Dict[str, float], doc_id: int) -> float:
        score = 0.0
        doc_tfidf = self.tfidf_matrix.get(doc_id, {})
        
        for term, query_score in query_tfidf.items():
            doc_score = doc_tfidf.get(term, 0.0)
            score += query_score * doc_score
        
        return score
    
    def search(self, query: str, top_k: int = 10) -> List[Dict]:
        query_terms = self.preprocessor.preprocess(query)
        
        if not query_terms:
            return []
        
        query_tfidf = self.calculate_query_tfidf(query_terms)
        doc_scores = []
        
        for doc_id in range(len(self.collection.documents)):
            score = self.calculate_similarity(query_tfidf, doc_id)
            if score > 0:
                doc_scores.append((doc_id, score))
        
        doc_scores.sort(key=lambda x: x[1], reverse=True)
        
        results = []
        for doc_id, score in doc_scores[:top_k]:
            doc = self.collection.get_document(doc_id)
            
            title = "N/A"
            if 'Heading' in doc['original']:
                title = doc['original']['Heading']
            elif 'heading' in doc['original']:
                title = doc['original']['heading']
            elif 'title' in doc['original']:
                title = doc['original']['title']
            
            results.append({
                'rank': len(results) + 1,
                'doc_id': doc_id,
                'score': score,
                'title': title,
                'text_preview': doc['text'][:200] + "..."
            })
        
        return results


# ============================================================
# PHASE 4: BM25 MODEL
# ============================================================

class BM25Model:
    """
    Implements BM25 Ranking Function
    
    Mathematical Formula:
    --------------------
    BM25(q, d) = Σ IDF(qi) × (f(qi,d) × (k1 + 1)) / 
                             (f(qi,d) + k1 × (1 - b + b × |d|/avgdl))
    
    Where:
    - q: query, d: document, qi: ith query term
    - f(qi,d): frequency of qi in document d
    - |d|: length of document d
    - avgdl: average document length
    - k1: term frequency saturation (typically 1.2 to 2.0)
    - b: length normalization (typically 0.75)
    """
    
    def __init__(self, 
                 collection: DocumentCollection, 
                 inverted_index: InvertedIndex,
                 k1: float = 1.5,
                 b: float = 0.75):
        """
        Initialize BM25 Model
        
        Args:
            k1: Controls term frequency saturation (1.2 to 2.0)
            b: Controls length normalization (0 to 1, default 0.75)
        """
        self.collection = collection
        self.index = inverted_index
        self.preprocessor = collection.preprocessor
        
        self.k1 = k1
        self.b = b
        
        self.num_documents = len(collection.documents)
        self.avgdl = 0.0
        self.doc_lengths = {}
        self.idf_values = {}
        self.term_frequencies = {}
        
        print("\n✓ BM25 Model initialized")
        print(f"  Parameters: k1={k1}, b={b}")
        
        self.calculate_statistics()
        self.calculate_idf_values()
    
    def calculate_statistics(self):
        """Calculate document statistics"""
        print("\n" + "="*60)
        print("CALCULATING DOCUMENT STATISTICS")
        print("="*60)
        
        total_length = 0
        
        for doc in self.collection.processed_docs:
            doc_id = doc['id']
            tokens = doc['tokens']
            
            doc_length = len(tokens)
            self.doc_lengths[doc_id] = doc_length
            total_length += doc_length
            
            self.term_frequencies[doc_id] = Counter(tokens)
            
            if (doc_id + 1) % 500 == 0:
                print(f"  Processed {doc_id + 1}/{self.num_documents} documents...")
        
        self.avgdl = total_length / self.num_documents if self.num_documents > 0 else 0
        
        print(f"\n✓ Statistics calculated")
        print(f"  Total documents: {self.num_documents}")
        print(f"  Average document length: {self.avgdl:.2f} terms")
        print(f"  Shortest document: {min(self.doc_lengths.values())} terms")
        print(f"  Longest document: {max(self.doc_lengths.values())} terms")
    
    def calculate_idf_values(self):
        """Calculate BM25 IDF values"""
        print("\n" + "="*60)
        print("CALCULATING BM25 IDF VALUES")
        print("="*60)
        
        all_terms = self.index.index.keys()
        
        for term in all_terms:
            df = self.index.get_term_frequency(term)
            # BM25 IDF formula
            idf = math.log((self.num_documents - df + 0.5) / (df + 0.5) + 1.0)
            self.idf_values[term] = idf
        
        print(f"✓ Calculated IDF for {len(self.idf_values)} unique terms")
        self.display_idf_statistics()
    
    def display_idf_statistics(self):
        """Display IDF statistics"""
        print("\n" + "-"*60)
        print("SAMPLE IDF VALUES")
        print("-"*60)
        
        sorted_idf = sorted(self.idf_values.items(), key=lambda x: x[1], reverse=True)
        
        print("\nTop 10 Most Discriminative Terms (Highest IDF):")
        for i, (term, idf) in enumerate(sorted_idf[:10], 1):
            df = self.index.get_term_frequency(term)
            print(f"{i:2d}. '{term}' - IDF: {idf:.4f} (in {df} docs)")
        
        print("\nTop 10 Most Common Terms (Lowest IDF):")
        for i, (term, idf) in enumerate(sorted_idf[-10:][::-1], 1):
            df = self.index.get_term_frequency(term)
            print(f"{i:2d}. '{term}' - IDF: {idf:.4f} (in {df} docs)")
    
    def calculate_bm25_score(self, query_terms: List[str], doc_id: int) -> float:
        """Calculate BM25 score for a document"""
        score = 0.0
        
        doc_length = self.doc_lengths.get(doc_id, 0)
        doc_term_freq = self.term_frequencies.get(doc_id, {})
        
        # Length normalization factor
        length_norm = 1 - self.b + self.b * (doc_length / self.avgdl)
        
        for term in query_terms:
            term_freq = doc_term_freq.get(term, 0)
            
            if term_freq == 0:
                continue
            
            idf = self.idf_values.get(term, 0)
            
            # BM25 formula
            numerator = term_freq * (self.k1 + 1)
            denominator = term_freq + self.k1 * length_norm
            term_score = idf * (numerator / denominator)
            
            score += term_score
        
        return score
    
    def search(self, query: str, top_k: int = 10, verbose: bool = True) -> List[Dict]:
        """Search using BM25 ranking"""
        if verbose:
            print("\n" + "="*60)
            print(f"EXECUTING BM25 QUERY: '{query}'")
            print("="*60)
        
        query_terms = self.preprocessor.preprocess(query)
        
        if verbose:
            print(f"\nPreprocessed query: {query_terms}")
        
        if not query_terms:
            if verbose:
                print("✗ No valid query terms")
            return []
        
        if verbose:
            print("\nRanking documents...")
        
        doc_scores = []
        
        for doc_id in range(self.num_documents):
            score = self.calculate_bm25_score(query_terms, doc_id)
            
            if score > 0:
                doc_scores.append((doc_id, score))
        
        doc_scores.sort(key=lambda x: x[1], reverse=True)
        
        if verbose:
            print(f"✓ Found {len(doc_scores)} relevant documents")
        
        results = []
        for doc_id, score in doc_scores[:top_k]:
            doc = self.collection.get_document(doc_id)
            
            title = "N/A"
            if 'Heading' in doc['original']:
                title = doc['original']['Heading']
            elif 'heading' in doc['original']:
                title = doc['original']['heading']
            elif 'title' in doc['original']:
                title = doc['original']['title']
            
            results.append({
                'rank': len(results) + 1,
                'doc_id': doc_id,
                'score': score,
                'title': title,
                'text_preview': doc['text'][:150] + "..."
            })
        
        return results
    
    def explain_score(self, query: str, doc_id: int):
        """Explain BM25 score calculation"""
        print("\n" + "="*60)
        print(f"BM25 SCORE EXPLANATION")
        print("="*60)
        
        query_terms = self.preprocessor.preprocess(query)
        
        print(f"\nQuery: '{query}'")
        print(f"Preprocessed: {query_terms}")
        print(f"\nDocument ID: {doc_id}")
        
        doc_length = self.doc_lengths.get(doc_id, 0)
        doc_term_freq = self.term_frequencies.get(doc_id, {})
        
        print(f"Document length: {doc_length} terms")
        print(f"Average length: {self.avgdl:.2f} terms")
        print(f"Length ratio: {doc_length/self.avgdl:.2f}")
        
        length_norm = 1 - self.b + self.b * (doc_length / self.avgdl)
        print(f"Length normalization: {length_norm:.4f}")
        
        print("\n" + "-"*60)
        print("TERM-BY-TERM BREAKDOWN")
        print("-"*60)
        
        total_score = 0.0
        
        for term in query_terms:
            term_freq = doc_term_freq.get(term, 0)
            idf = self.idf_values.get(term, 0)
            df = self.index.get_term_frequency(term)
            
            print(f"\nTerm: '{term}'")
            print(f"  TF in doc: {term_freq}")
            print(f"  DF: {df} docs")
            print(f"  IDF: {idf:.4f}")
            
            if term_freq > 0:
                numerator = term_freq * (self.k1 + 1)
                denominator = term_freq + self.k1 * length_norm
                term_score = idf * (numerator / denominator)
                
                print(f"  Score: {term_score:.4f}")
                total_score += term_score
            else:
                print(f"  Score: 0.0 (not in doc)")
        
        print("\n" + "-"*60)
        print(f"TOTAL BM25 SCORE: {total_score:.4f}")
        print("-"*60)


def compare_models(collection, inverted_index, query: str):
    """Compare TF-IDF and BM25"""
    print("\n" + "="*60)
    print("MODEL COMPARISON: TF-IDF vs BM25")
    print("="*60)
    print(f"\nQuery: '{query}'")
    
    print("\nInitializing models...")
    tfidf_model = TFIDFModel(collection, inverted_index)
    bm25_model = BM25Model(collection, inverted_index)
    
    print("\nRetrieving results...")
    tfidf_results = tfidf_model.search(query, top_k=5)
    bm25_results = bm25_model.search(query, top_k=5, verbose=False)
    
    print("\n" + "="*60)
    print("TOP 5 RESULTS COMPARISON")
    print("="*60)
    
    print(f"\n{'Rank':<6} {'Model':<10} {'Score':<12} {'Doc ID':<10} {'Title':<40}")
    print("-" * 80)
    
    for i in range(min(5, len(tfidf_results), len(bm25_results))):
        tfidf_r = tfidf_results[i]
        bm25_r = bm25_results[i]
        
        print(f"{i+1:<6} {'TF-IDF':<10} {tfidf_r['score']:<12.4f} {tfidf_r['doc_id']:<10} {tfidf_r['title'][:40]}")
        print(f"{'':<6} {'BM25':<10} {bm25_r['score']:<12.4f} {bm25_r['doc_id']:<10} {bm25_r['title'][:40]}")
        print()
    
    tfidf_doc_ids = set([r['doc_id'] for r in tfidf_results])
    bm25_doc_ids = set([r['doc_id'] for r in bm25_results])
    common_docs = tfidf_doc_ids & bm25_doc_ids
    
    print("\n" + "="*60)
    print("ANALYSIS")
    print("="*60)
    print(f"Documents in both top-5: {len(common_docs)}/5")
    print(f"\nKey Differences:")
    print("  TF-IDF: Linear term frequency, weak length norm")
    print("  BM25: Saturation effect, strong length norm")


def demonstrate_bm25():
    """Complete BM25 demonstration"""
    print("\n" + "="*60)
    print("INFORMATION RETRIEVAL SYSTEM - PHASE 4")
    print("BM25 RETRIEVAL MODEL")
    print("="*60)
    
    CSV_FILE_PATH = r"C:\Users\lenovo\news_articles.csv"
    
    print("\nStep 1: Loading documents...")
    collection = DocumentCollection(CSV_FILE_PATH)
    
    print("\nStep 2: Building inverted index...")
    inverted_index = InvertedIndex()
    inverted_index.build_index(collection.processed_docs)
    
    print("\nStep 3: Initializing BM25 Model...")
    bm25_model = BM25Model(collection, inverted_index, k1=1.5, b=0.75)
    
    print("\n" + "="*60)
    print("QUERY DEMONSTRATIONS")
    print("="*60)
    
    queries = [
        "karachi transport government",
        "pakistan cricket match",
        "lahore education university",
    ]
    
    for query in queries:
        results = bm25_model.search(query, top_k=5)
        
        print("\n" + "="*60)
        print(f"Query: '{query}'")
        print("="*60)
        
        for result in results:
            print(f"\n{result['rank']}. [BM25: {result['score']:.4f}] Doc {result['doc_id']}")
            print(f"   {result['title']}")
            print(f"   {result['text_preview']}")
    
    # Explain first result
    if len(queries) > 0:
        results = bm25_model.search(queries[0], top_k=1, verbose=False)
        if results:
            bm25_model.explain_score(queries[0], results[0]['doc_id'])
    
    # Compare models
    print("\n" + "="*60)
    print("COMPARING WITH TF-IDF")
    print("="*60)
    compare_models(collection, inverted_index, queries[0])
    
    print("\n" + "="*60)
    print("PHASE 4 COMPLETE!")
    print("="*60)
    print("\n✓ Implemented BM25 ranking")
    print("✓ Term frequency saturation")
    print("✓ Document length normalization")
    print("✓ Compared with TF-IDF")
    print("\nBM25 is the gold standard for text retrieval!")
    print("="*60)


if __name__ == "__main__":
    demonstrate_bm25()

Checking NLTK resources...
✓ NLTK resources ready

INFORMATION RETRIEVAL SYSTEM - PHASE 4
BM25 RETRIEVAL MODEL

Step 1: Loading documents...
✓ Loaded with latin-1 encoding
✓ Loaded 2692 documents

Preprocessing documents...
  Processed 500/2692 documents...
  Processed 1000/2692 documents...
  Processed 1500/2692 documents...
  Processed 2000/2692 documents...
  Processed 2500/2692 documents...
✓ All 2692 documents preprocessed!

Step 2: Building inverted index...

Building inverted index...
  Indexed 500/2692 documents...
  Indexed 1000/2692 documents...
  Indexed 1500/2692 documents...
  Indexed 2000/2692 documents...
  Indexed 2500/2692 documents...
✓ Indexed 18196 unique terms

Step 3: Initializing BM25 Model...

✓ BM25 Model initialized
  Parameters: k1=1.5, b=0.75

CALCULATING DOCUMENT STATISTICS
  Processed 500/2692 documents...
  Processed 1000/2692 documents...
  Processed 1500/2692 documents...
  Processed 2000/2692 documents...
  Processed 2500/2692 documents...

✓ Statistic

In [5]:
"""
INFORMATION RETRIEVAL SYSTEM - PHASE 5: VECTOR SPACE MODEL
===========================================================
This implements the Vector Space Model (VSM) with Cosine Similarity.

What is the Vector Space Model?
--------------------------------
VSM represents documents and queries as vectors in a high-dimensional space,
where each dimension corresponds to a term in the vocabulary.

Key Concept: GEOMETRIC INTERPRETATION
- Documents are points in space
- Similar documents are close together
- We measure "closeness" using angles (cosine similarity)

Why Cosine Similarity?
----------------------
Cosine measures the ANGLE between vectors, not distance.
- Angle of 0° (cos=1): Identical documents
- Angle of 90° (cos=0): Unrelated documents
- Length-independent: Fair comparison of all document sizes

Author: Saad Ali
Course: Information Retrieval
"""

import pandas as pd
import numpy as np
import math
import re
import nltk
from collections import defaultdict, Counter
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from typing import List, Dict, Tuple, Set

# Download required NLTK data
print("Checking NLTK resources...")
try:
    nltk.data.find('tokenizers/punkt_tab')
    nltk.data.find('corpora/stopwords')
    print("✓ NLTK resources ready")
except LookupError:
    print("Downloading required NLTK data...")
    nltk.download('punkt_tab')
    nltk.download('stopwords')


# ============================================================
# PREVIOUS PHASES (Embedded)
# ============================================================

class DocumentPreprocessor:
    """Handles all text preprocessing tasks"""
    
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        self.stemmer = PorterStemmer()
    
    def clean_text(self, text):
        if not isinstance(text, str):
            return ""
        text = text.lower()
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)
        text = re.sub(r'\S+@\S+', '', text)
        text = re.sub(r'[^a-zA-Z\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    
    def tokenize(self, text):
        return word_tokenize(text)
    
    def remove_stopwords(self, tokens):
        return [word for word in tokens if word not in self.stop_words]
    
    def stem_tokens(self, tokens):
        return [self.stemmer.stem(word) for word in tokens]
    
    def preprocess(self, text):
        cleaned = self.clean_text(text)
        tokens = self.tokenize(cleaned)
        tokens = self.remove_stopwords(tokens)
        tokens = self.stem_tokens(tokens)
        tokens = [token for token in tokens if len(token) > 1]
        return tokens


class DocumentCollection:
    """Manages the entire document collection"""
    
    def __init__(self, csv_path):
        self.preprocessor = DocumentPreprocessor()
        self.documents = []
        self.processed_docs = []
        self.doc_ids = []
        self.load_documents(csv_path)
        
    def load_documents(self, csv_path):
        try:
            encodings_to_try = ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252']
            df = None
            
            for encoding in encodings_to_try:
                try:
                    df = pd.read_csv(csv_path, encoding=encoding)
                    print(f"✓ Loaded with {encoding} encoding")
                    break
                except UnicodeDecodeError:
                    continue
            
            if df is None:
                raise ValueError("Could not read file")
            
            print(f"✓ Loaded {len(df)} documents")
            
            for idx, row in df.iterrows():
                doc_text = ""
                
                if 'Heading' in df.columns:
                    doc_text += str(row['Heading']) + " "
                elif 'heading' in df.columns:
                    doc_text += str(row['heading']) + " "
                elif 'title' in df.columns:
                    doc_text += str(row['title']) + " "
                
                if 'Article' in df.columns:
                    doc_text += str(row['Article'])
                elif 'article' in df.columns:
                    doc_text += str(row['article'])
                elif 'content' in df.columns:
                    doc_text += str(row['content'])
                elif 'text' in df.columns:
                    doc_text += str(row['text'])
                
                self.documents.append({
                    'id': idx,
                    'text': doc_text,
                    'original': row.to_dict()
                })
                self.doc_ids.append(idx)
            
            self.preprocess_collection()
            
        except Exception as e:
            print(f"✗ Error loading documents: {e}")
    
    def preprocess_collection(self):
        print("\nPreprocessing documents...")
        for idx, doc in enumerate(self.documents):
            processed = self.preprocessor.preprocess(doc['text'])
            self.processed_docs.append({
                'id': doc['id'],
                'tokens': processed,
                'token_count': len(processed)
            })
            
            if (idx + 1) % 500 == 0:
                print(f"  Processed {idx + 1}/{len(self.documents)} documents...")
        
        print(f"✓ All {len(self.processed_docs)} documents preprocessed!")
    
    def get_document(self, doc_id):
        return self.documents[doc_id]
    
    def get_processed_document(self, doc_id):
        return self.processed_docs[doc_id]


class InvertedIndex:
    """Inverted Index for efficient retrieval"""
    
    def __init__(self):
        self.index = defaultdict(set)
        self.term_doc_freq = defaultdict(int)
        self.num_documents = 0
    
    def build_index(self, processed_docs: List[Dict]):
        print("\nBuilding inverted index...")
        
        self.num_documents = len(processed_docs)
        
        for doc in processed_docs:
            doc_id = doc['id']
            tokens = doc['tokens']
            unique_terms = set(tokens)
            
            for term in unique_terms:
                self.index[term].add(doc_id)
                self.term_doc_freq[term] += 1
            
            if (doc_id + 1) % 500 == 0:
                print(f"  Indexed {doc_id + 1}/{self.num_documents} documents...")
        
        print(f"✓ Indexed {len(self.index)} unique terms")
    
    def get_postings(self, term: str) -> Set[int]:
        return self.index.get(term, set())
    
    def get_term_frequency(self, term: str) -> int:
        return self.term_doc_freq.get(term, 0)


class TFIDFModel:
    """TF-IDF Model (for comparison)"""
    
    def __init__(self, collection: DocumentCollection, inverted_index: InvertedIndex):
        self.collection = collection
        self.index = inverted_index
        self.preprocessor = collection.preprocessor
        self.tfidf_matrix = {}
        self.idf_values = {}
        self.term_frequencies = {}
        self.build_tfidf_matrix()
    
    def calculate_tf(self, term_count: int, total_terms: int) -> float:
        if term_count == 0:
            return 0.0
        return term_count / total_terms if total_terms > 0 else 0.0
    
    def calculate_idf(self, term: str) -> float:
        N = self.index.num_documents
        df = self.index.get_term_frequency(term)
        if df == 0:
            return 0.0
        return math.log(N / (1 + df))
    
    def build_tfidf_matrix(self):
        print("\nBuilding TF-IDF matrix (for comparison)...")
        
        for doc in self.collection.processed_docs:
            doc_id = doc['id']
            self.term_frequencies[doc_id] = Counter(doc['tokens'])
        
        for term in self.index.index.keys():
            self.idf_values[term] = self.calculate_idf(term)
        
        for doc in self.collection.processed_docs:
            doc_id = doc['id']
            total_terms = doc['token_count']
            self.tfidf_matrix[doc_id] = {}
            
            for term, count in self.term_frequencies[doc_id].items():
                tf = self.calculate_tf(count, total_terms)
                idf = self.idf_values.get(term, 0.0)
                self.tfidf_matrix[doc_id][term] = tf * idf
        
        print("✓ TF-IDF matrix built")
    
    def calculate_query_tfidf(self, query_terms: List[str]) -> Dict[str, float]:
        query_tfidf = {}
        query_tf = Counter(query_terms)
        total = len(query_terms)
        
        for term, count in query_tf.items():
            tf = self.calculate_tf(count, total)
            idf = self.idf_values.get(term, 0.0)
            query_tfidf[term] = tf * idf
        
        return query_tfidf
    
    def calculate_similarity(self, query_tfidf: Dict[str, float], doc_id: int) -> float:
        score = 0.0
        doc_tfidf = self.tfidf_matrix.get(doc_id, {})
        
        for term, q_score in query_tfidf.items():
            score += q_score * doc_tfidf.get(term, 0.0)
        
        return score
    
    def search(self, query: str, top_k: int = 10) -> List[Dict]:
        query_terms = self.preprocessor.preprocess(query)
        if not query_terms:
            return []
        
        query_tfidf = self.calculate_query_tfidf(query_terms)
        doc_scores = []
        
        for doc_id in range(len(self.collection.documents)):
            score = self.calculate_similarity(query_tfidf, doc_id)
            if score > 0:
                doc_scores.append((doc_id, score))
        
        doc_scores.sort(key=lambda x: x[1], reverse=True)
        
        results = []
        for doc_id, score in doc_scores[:top_k]:
            doc = self.collection.get_document(doc_id)
            title = doc['original'].get('Heading', doc['original'].get('title', 'N/A'))
            
            results.append({
                'rank': len(results) + 1,
                'doc_id': doc_id,
                'score': score,
                'title': title,
                'text_preview': doc['text'][:150] + "..."
            })
        
        return results


class BM25Model:
    """BM25 Model (for comparison)"""
    
    def __init__(self, collection: DocumentCollection, inverted_index: InvertedIndex, k1=1.5, b=0.75):
        self.collection = collection
        self.index = inverted_index
        self.preprocessor = collection.preprocessor
        self.k1 = k1
        self.b = b
        self.num_documents = len(collection.documents)
        self.avgdl = 0.0
        self.doc_lengths = {}
        self.idf_values = {}
        self.term_frequencies = {}
        self.calculate_statistics()
        self.calculate_idf_values()
    
    def calculate_statistics(self):
        print("\nCalculating BM25 statistics...")
        total_length = 0
        
        for doc in self.collection.processed_docs:
            doc_id = doc['id']
            doc_length = len(doc['tokens'])
            self.doc_lengths[doc_id] = doc_length
            total_length += doc_length
            self.term_frequencies[doc_id] = Counter(doc['tokens'])
        
        self.avgdl = total_length / self.num_documents if self.num_documents > 0 else 0
        print(f"✓ BM25 statistics calculated (avgdl: {self.avgdl:.2f})")
    
    def calculate_idf_values(self):
        for term in self.index.index.keys():
            df = self.index.get_term_frequency(term)
            idf = math.log((self.num_documents - df + 0.5) / (df + 0.5) + 1.0)
            self.idf_values[term] = idf
    
    def calculate_bm25_score(self, query_terms: List[str], doc_id: int) -> float:
        score = 0.0
        doc_length = self.doc_lengths.get(doc_id, 0)
        doc_term_freq = self.term_frequencies.get(doc_id, {})
        length_norm = 1 - self.b + self.b * (doc_length / self.avgdl)
        
        for term in query_terms:
            term_freq = doc_term_freq.get(term, 0)
            if term_freq == 0:
                continue
            
            idf = self.idf_values.get(term, 0)
            numerator = term_freq * (self.k1 + 1)
            denominator = term_freq + self.k1 * length_norm
            score += idf * (numerator / denominator)
        
        return score
    
    def search(self, query: str, top_k: int = 10, verbose: bool = False) -> List[Dict]:
        query_terms = self.preprocessor.preprocess(query)
        if not query_terms:
            return []
        
        doc_scores = []
        for doc_id in range(self.num_documents):
            score = self.calculate_bm25_score(query_terms, doc_id)
            if score > 0:
                doc_scores.append((doc_id, score))
        
        doc_scores.sort(key=lambda x: x[1], reverse=True)
        
        results = []
        for doc_id, score in doc_scores[:top_k]:
            doc = self.collection.get_document(doc_id)
            title = doc['original'].get('Heading', doc['original'].get('title', 'N/A'))
            
            results.append({
                'rank': len(results) + 1,
                'doc_id': doc_id,
                'score': score,
                'title': title,
                'text_preview': doc['text'][:150] + "..."
            })
        
        return results


# ============================================================
# PHASE 5: VECTOR SPACE MODEL
# ============================================================

class VectorSpaceModel:
    """
    Implements Vector Space Model with Cosine Similarity
    
    Mathematical Formula:
    --------------------
    Cosine Similarity = (q · d) / (||q|| × ||d||)
    
    Where:
    - q · d = dot product (Σ qi × di)
    - ||q|| = magnitude of query vector (√Σ qi²)
    - ||d|| = magnitude of document vector (√Σ di²)
    """
    
    def __init__(self, collection: DocumentCollection, inverted_index: InvertedIndex):
        self.collection = collection
        self.index = inverted_index
        self.preprocessor = collection.preprocessor
        
        # Document vectors: {doc_id: {term: weight}}
        self.doc_vectors = {}
        
        # Document magnitudes: {doc_id: magnitude}
        self.doc_magnitudes = {}
        
        # IDF values
        self.idf_values = {}
        
        # Term frequencies
        self.term_frequencies = {}
        
        # Vocabulary
        self.vocabulary = set()
        
        print("\n✓ Vector Space Model initialized")
        self.build_document_vectors()
    
    def calculate_idf(self, term: str) -> float:
        """Calculate IDF value"""
        N = self.index.num_documents
        df = self.index.get_term_frequency(term)
        
        if df == 0:
            return 0.0
        
        return math.log((N + 1) / (df + 1)) + 1
    
    def calculate_tf(self, term_count: int, total_terms: int) -> float:
        """Calculate normalized TF"""
        if total_terms == 0:
            return 0.0
        return term_count / total_terms
    
    def build_document_vectors(self):
        """Build TF-IDF vectors for all documents"""
        print("\n" + "="*60)
        print("BUILDING DOCUMENT VECTORS")
        print("="*60)
        
        # Step 1: Calculate term frequencies
        print("\nStep 1: Calculating term frequencies...")
        for doc in self.collection.processed_docs:
            doc_id = doc['id']
            tokens = doc['tokens']
            
            self.term_frequencies[doc_id] = Counter(tokens)
            self.vocabulary.update(tokens)
            
            if (doc_id + 1) % 500 == 0:
                print(f"  Processed {doc_id + 1}/{len(self.collection.processed_docs)} documents...")
        
        print(f"\n✓ Vocabulary size: {len(self.vocabulary)} unique terms")
        
        # Step 2: Calculate IDF values
        print("\nStep 2: Calculating IDF values...")
        for term in self.vocabulary:
            self.idf_values[term] = self.calculate_idf(term)
        
        print(f"✓ Calculated IDF for {len(self.idf_values)} terms")
        
        # Step 3: Build TF-IDF vectors
        print("\nStep 3: Building TF-IDF vectors...")
        for doc in self.collection.processed_docs:
            doc_id = doc['id']
            total_terms = doc['token_count']
            
            vector = {}
            
            for term, count in self.term_frequencies[doc_id].items():
                tf = self.calculate_tf(count, total_terms)
                idf = self.idf_values[term]
                tfidf = tf * idf
                
                if tfidf > 0:
                    vector[term] = tfidf
            
            self.doc_vectors[doc_id] = vector
            
            # Calculate magnitude
            magnitude = self.calculate_magnitude(vector)
            self.doc_magnitudes[doc_id] = magnitude
            
            if (doc_id + 1) % 500 == 0:
                print(f"  Built vectors for {doc_id + 1}/{len(self.collection.processed_docs)} documents...")
        
        print("\n✓ Document vectors built successfully!")
        self.display_vector_statistics()
    
    def calculate_magnitude(self, vector: Dict[str, float]) -> float:
        """Calculate vector magnitude: sqrt(Σ wi²)"""
        sum_of_squares = sum(weight ** 2 for weight in vector.values())
        return math.sqrt(sum_of_squares)
    
    def calculate_dot_product(self, vector1: Dict[str, float], vector2: Dict[str, float]) -> float:
        """Calculate dot product: Σ (v1_i × v2_i)"""
        dot_product = 0.0
        
        # Iterate over smaller vector
        if len(vector1) < len(vector2):
            smaller, larger = vector1, vector2
        else:
            smaller, larger = vector2, vector1
        
        for term, weight1 in smaller.items():
            if term in larger:
                dot_product += weight1 * larger[term]
        
        return dot_product
    
    def calculate_cosine_similarity(self, query_vector: Dict[str, float], doc_id: int) -> float:
        """
        Calculate cosine similarity
        
        Cosine = (q · d) / (||q|| × ||d||)
        """
        doc_vector = self.doc_vectors.get(doc_id, {})
        doc_magnitude = self.doc_magnitudes.get(doc_id, 0)
        
        if doc_magnitude == 0:
            return 0.0
        
        query_magnitude = self.calculate_magnitude(query_vector)
        
        if query_magnitude == 0:
            return 0.0
        
        dot_product = self.calculate_dot_product(query_vector, doc_vector)
        cosine_sim = dot_product / (query_magnitude * doc_magnitude)
        
        return cosine_sim
    
    def build_query_vector(self, query_terms: List[str]) -> Dict[str, float]:
        """Build TF-IDF vector for query"""
        query_vector = {}
        query_tf = Counter(query_terms)
        total = len(query_terms)
        
        for term, count in query_tf.items():
            tf = self.calculate_tf(count, total)
            idf = self.idf_values.get(term, 0)
            tfidf = tf * idf
            
            if tfidf > 0:
                query_vector[term] = tfidf
        
        return query_vector
    
    def search(self, query: str, top_k: int = 10, verbose: bool = True) -> List[Dict]:
        """Search using Vector Space Model with Cosine Similarity"""
        if verbose:
            print("\n" + "="*60)
            print(f"EXECUTING VSM QUERY: '{query}'")
            print("="*60)
        
        query_terms = self.preprocessor.preprocess(query)
        
        if verbose:
            print(f"\nPreprocessed query: {query_terms}")
        
        if not query_terms:
            if verbose:
                print("✗ No valid query terms")
            return []
        
        query_vector = self.build_query_vector(query_terms)
        
        if verbose:
            print("\nQuery vector (top terms):")
            sorted_qv = sorted(query_vector.items(), key=lambda x: x[1], reverse=True)[:5]
            for term, weight in sorted_qv:
                print(f"  '{term}': {weight:.4f}")
        
        if verbose:
            print("\nCalculating cosine similarities...")
        
        doc_scores = []
        
        for doc_id in range(len(self.collection.documents)):
            similarity = self.calculate_cosine_similarity(query_vector, doc_id)
            
            if similarity > 0:
                doc_scores.append((doc_id, similarity))
        
        doc_scores.sort(key=lambda x: x[1], reverse=True)
        
        if verbose:
            print(f"✓ Found {len(doc_scores)} relevant documents")
        
        results = []
        for doc_id, similarity in doc_scores[:top_k]:
            doc = self.collection.get_document(doc_id)
            title = doc['original'].get('Heading', doc['original'].get('title', 'N/A'))
            
            results.append({
                'rank': len(results) + 1,
                'doc_id': doc_id,
                'score': similarity,
                'title': title,
                'text_preview': doc['text'][:150] + "..."
            })
        
        return results
    
    def display_vector_statistics(self):
        """Display statistics about vectors"""
        print("\n" + "="*60)
        print("VECTOR SPACE STATISTICS")
        print("="*60)
        
        total_dims = len(self.vocabulary)
        avg_nonzero = np.mean([len(vec) for vec in self.doc_vectors.values()])
        sparsity = (1 - avg_nonzero / total_dims) * 100
        
        print(f"\nVocabulary (dimensions): {total_dims}")
        print(f"Avg non-zero dims per doc: {avg_nonzero:.2f}")
        print(f"Vector sparsity: {sparsity:.2f}%")
        
        magnitudes = list(self.doc_magnitudes.values())
        print(f"\nDocument magnitudes:")
        print(f"  Average: {np.mean(magnitudes):.4f}")
        print(f"  Min: {np.min(magnitudes):.4f}")
        print(f"  Max: {np.max(magnitudes):.4f}")
    
    def explain_similarity(self, query: str, doc_id: int):
        """Explain cosine similarity calculation"""
        print("\n" + "="*60)
        print("COSINE SIMILARITY EXPLANATION")
        print("="*60)
        
        query_terms = self.preprocessor.preprocess(query)
        query_vector = self.build_query_vector(query_terms)
        
        print(f"\nQuery: '{query}'")
        print(f"Preprocessed: {query_terms}")
        print(f"Document ID: {doc_id}")
        
        doc_vector = self.doc_vectors.get(doc_id, {})
        
        print("\n" + "-"*60)
        print("QUERY VECTOR:")
        for term, weight in sorted(query_vector.items(), key=lambda x: x[1], reverse=True):
            print(f"  '{term}': {weight:.4f}")
        
        print("\nDOCUMENT VECTOR (matching terms):")
        for term in query_terms:
            doc_weight = doc_vector.get(term, 0.0)
            print(f"  '{term}': {doc_weight:.4f}")
        
        query_magnitude = self.calculate_magnitude(query_vector)
        doc_magnitude = self.doc_magnitudes.get(doc_id, 0)
        dot_product = self.calculate_dot_product(query_vector, doc_vector)
        cosine_sim = self.calculate_cosine_similarity(query_vector, doc_id)
        
        print("\n" + "-"*60)
        print("CALCULATION:")
        print(f"  Dot product (q · d): {dot_product:.4f}")
        print(f"  Query magnitude ||q||: {query_magnitude:.4f}")
        print(f"  Doc magnitude ||d||: {doc_magnitude:.4f}")
        print(f"  Cosine similarity: {dot_product:.4f} / ({query_magnitude:.4f} × {doc_magnitude:.4f})")
        print(f"  Result: {cosine_sim:.4f}")
        
        print("\n" + "-"*60)
        if cosine_sim >= 0.7:
            print(f"✓ High similarity ({cosine_sim:.4f}) - Very relevant!")
        elif cosine_sim >= 0.3:
            print(f"○ Moderate similarity ({cosine_sim:.4f}) - Somewhat relevant")
        else:
            print(f"✗ Low similarity ({cosine_sim:.4f}) - Not very relevant")


def compare_all_models(collection, inverted_index, query: str):
    """Compare all models"""
    print("\n" + "="*60)
    print("MODEL COMPARISON: TF-IDF vs BM25 vs VSM")
    print("="*60)
    print(f"\nQuery: '{query}'")
    
    print("\nInitializing models...")
    tfidf_model = TFIDFModel(collection, inverted_index)
    bm25_model = BM25Model(collection, inverted_index)
    vsm_model = VectorSpaceModel(collection, inverted_index)
    
    print("\nRetrieving results...")
    tfidf_results = tfidf_model.search(query, top_k=5)
    bm25_results = bm25_model.search(query, top_k=5, verbose=False)
    vsm_results = vsm_model.search(query, top_k=5, verbose=False)
    
    print("\n" + "="*60)
    print("TOP 5 RESULTS")
    print("="*60)
    
    print(f"\n{'Rank':<6} {'TF-IDF':<12} {'BM25':<12} {'VSM (Cosine)':<15}")
    print("-" * 50)
    
    for i in range(5):
        print(f"{i+1:<6}", end="")
        
        if i < len(tfidf_results):
            print(f"{tfidf_results[i]['score']:>10.4f}  ", end="")
        else:
            print(f"{'N/A':>10}  ", end="")
        
        if i < len(bm25_results):
            print(f"{bm25_results[i]['score']:>10.4f}  ", end="")
        else:
            print(f"{'N/A':>10}  ", end="")
        
        if i < len(vsm_results):
            print(f"{vsm_results[i]['score']:>13.4f}")
        else:
            print(f"{'N/A':>13}")
    
    print("\n" + "="*60)
    print("MODEL CHARACTERISTICS")
    print("="*60)
    print("\nTF-IDF: Simple, linear, no normalization")
    print("BM25: Saturation, length norm, industry standard")
    print("VSM: Geometric, angle-based, intuitive")


def demonstrate_vsm():
    """Complete VSM demonstration"""
    print("\n" + "="*60)
    print("INFORMATION RETRIEVAL SYSTEM - PHASE 5")
    print("VECTOR SPACE MODEL WITH COSINE SIMILARITY")
    print("="*60)
    
    CSV_FILE_PATH = r"C:\Users\lenovo\news_articles.csv"
    
    print("\nLoading documents...")
    collection = DocumentCollection(CSV_FILE_PATH)
    
    print("\nBuilding inverted index...")
    inverted_index = InvertedIndex()
    inverted_index.build_index(collection.processed_docs)
    
    print("\nInitializing Vector Space Model...")
    vsm_model = VectorSpaceModel(collection, inverted_index)
    
    print("\n" + "="*60)
    print("QUERY DEMONSTRATIONS")
    print("="*60)
    
    queries = [
        "karachi transport government",
        "pakistan cricket match",
        "lahore education university"
    ]
    
    for query in queries:
        results = vsm_model.search(query, top_k=5)
        
        print("\n" + "="*60)
        print(f"Query: '{query}'")
        print("="*60)
        
        for result in results:
            print(f"\n{result['rank']}. [Cosine: {result['score']:.4f}] Doc {result['doc_id']}")
            print(f"   {result['title']}")
            print(f"   {result['text_preview']}")
    
    # Explain similarity
    if len(queries) > 0:
        print("\n" + "="*60)
        print("SIMILARITY EXPLANATION EXAMPLE")
        print("="*60)
        results = vsm_model.search(queries[0], top_k=1, verbose=False)
        if results:
            vsm_model.explain_similarity(queries[0], results[0]['doc_id'])
    
    # Compare all models
    print("\n" + "="*60)
    print("COMPARING ALL MODELS")
    print("="*60)
    compare_all_models(collection, inverted_index, queries[0])
    
    print("\n" + "="*60)
    print("PHASE 5 COMPLETE!")
    print("="*60)
    print("\n✓ Implemented Vector Space Model")
    print("✓ Represented documents as vectors")
    print("✓ Implemented cosine similarity")
    print("✓ Compared all three retrieval models")
    print("\nVSM Advantages:")
    print("✓ Geometric interpretation (intuitive)")
    print("✓ Length-independent by design")
    print("✓ Solid theoretical foundation")
    print("✓ Extensible to clustering & classification")
    print("\nYou now have 3 powerful retrieval models!")
    print("="*60)


if __name__ == "__main__":
    demonstrate_vsm()

Checking NLTK resources...
✓ NLTK resources ready

INFORMATION RETRIEVAL SYSTEM - PHASE 5
VECTOR SPACE MODEL WITH COSINE SIMILARITY

Loading documents...
✓ Loaded with latin-1 encoding
✓ Loaded 2692 documents

Preprocessing documents...
  Processed 500/2692 documents...
  Processed 1000/2692 documents...
  Processed 1500/2692 documents...
  Processed 2000/2692 documents...
  Processed 2500/2692 documents...
✓ All 2692 documents preprocessed!

Building inverted index...

Building inverted index...
  Indexed 500/2692 documents...
  Indexed 1000/2692 documents...
  Indexed 1500/2692 documents...
  Indexed 2000/2692 documents...
  Indexed 2500/2692 documents...
✓ Indexed 18196 unique terms

Initializing Vector Space Model...

✓ Vector Space Model initialized

BUILDING DOCUMENT VECTORS

Step 1: Calculating term frequencies...
  Processed 500/2692 documents...
  Processed 1000/2692 documents...
  Processed 1500/2692 documents...
  Processed 2000/2692 documents...
  Processed 2500/2692 docume